In [3]:
pip install tensorflow_datasets

  Obtaining dependency information for tensorflow_datasets from https://files.pythonhosted.org/packages/a1/73/7a9ed7935f6833d73b32f1e2a1210082f5ccb95445440b4e2b0f66ab7792/tensorflow_datasets-4.9.3-py3-none-any.whl.metadata
  Obtaining dependency information for array-record from https://files.pythonhosted.org/packages/93/8e/c6b646029a9c544ecd6806c6ea0efb67d0ba3300e1d68518f58d9774d3f0/array_record-0.4.1-py310-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 5.3 MB/s eta 0:00:00
  Obtaining dependency information for etils[enp,epath,etree]>=0.9.0 from https://files.pythonhosted.org/packages/4a/6a/d58ec120f5e4babbf5001c144266ba623dcdae8e81dc6cdb422a98d0e0ce/etils-1.4.1-py3-none-any.whl.metadata
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for tensorflow-metadata from https://files.pythonhosted.org/packages/41/23/3705c7139886c079ef4c0e3be56a5a1fb90e9ee413a4b7caaee0ee0ea6fe/tensorflow_metadata-1.14.0-py3-none-any.whl.metadata
 

In [4]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_datasets as tfds
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Load the Food101 dataset
(ds_train, ds_test), ds_info = tfds.load('food101', split=['train[:80%]', 'train[80%:]'], as_supervised=True, with_info=True)



2023-09-18 13:51:15.736624: W tensorflow/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

...

Generating train examples...:   0%|          | 0/75750 [00:00<?, ? examples/s]

Shuffling /Users/sadianasrintisha/tensorflow_datasets/food101/2.0.0.incomplete07OA0P/food101-train.tfrecord*..…

Generating validation examples...:   0%|          | 0/25250 [00:00<?, ? examples/s]

Shuffling /Users/sadianasrintisha/tensorflow_datasets/food101/2.0.0.incomplete07OA0P/food101-validation.tfreco…

Dataset food101 downloaded and prepared to /Users/sadianasrintisha/tensorflow_datasets/food101/2.0.0. Subsequent calls will reuse this data.


In [5]:
# Define a function to resize images
def resize_image(image, label):
    image = tf.image.resize(image, (32, 32))
    return image, label

# Apply resizing to the dataset
ds_train = ds_train.map(resize_image)
ds_test = ds_test.map(resize_image)

# Normalize pixel values to be in the range [0, 1]
ds_train = ds_train.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))
ds_test = ds_test.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))

# Shuffle and batch the datasets
ds_train = ds_train.shuffle(10000).batch(64)
ds_test = ds_test.batch(64)

# Convert labels to categorical format
def categorical_labels(images, labels):
    return images, tf.one_hot(labels, depth=101)  # 101 classes in Food101 dataset

ds_train = ds_train.map(categorical_labels)
ds_test = ds_test.map(categorical_labels)

# Verify the shapes
for images, labels in ds_train.take(1):
    print("Images shape:", images.shape)
    print("Labels shape:", labels.shape)

for images, labels in ds_test.take(1):
    print("Images shape:", images.shape)
    print("Labels shape:", labels.shape)



Images shape: (64, 32, 32, 3)
Labels shape: (64, 101)
Images shape: (64, 32, 32, 3)
Labels shape: (64, 101)


In [12]:
from tensorflow.keras.optimizers.legacy import Adam as LegacyAdam



# Create the ResNet model without the top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
optimizer = LegacyAdam(learning_rate=0.001)
# Add the top layers for classification
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(101, activation='softmax'))  # Change the number of units to 101 for Food101 dataset
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])



In [ ]:
model.fit(ds_train, batch_size=64, epochs=50)

Epoch 1/50
947/947 [==============================] - 1060s 1s/step - loss: 4.8592 - accuracy: 0.0270
Epoch 2/50
947/947 [==============================] - 1494s 2s/step - loss: 4.2996 - accuracy: 0.0606
Epoch 3/50
947/947 [==============================] - 2924s 3s/step - loss: 3.9425 - accuracy: 0.1020
Epoch 4/50
947/947 [==============================] - 1255s 1s/step - loss: 3.7001 - accuracy: 0.1393
Epoch 5/50
947/947 [==============================] - 1326s 1s/step - loss: 3.5083 - accuracy: 0.1720
Epoch 6/50
883/947 [==========================>...] - ETA: 1:37 - loss: 3.3498 - accuracy: 0.2005

In [11]:
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix

# Make predictions on the test dataset
predictions = model.predict(ds_test)

# Convert one-hot encoded predictions back to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Manually generate ground truth labels for evaluation
true_labels = []
for _, label in ds_test:
    true_labels.extend(label.numpy())

true_labels = np.argmax(true_labels, axis=1)

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate F1 score
f1 = f1_score(true_labels, predicted_labels, average='weighted')

# Calculate confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Assuming you have test_loss and test_accuracy available
# If not, you'll need to calculate them based on your model and test dataset
print(f"Test Accuracy: {accuracy}")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

237/237 [==============================] - 15s 63ms/step
Test Accuracy: 0.01353135313531353
Accuracy: 0.01353135313531353
F1 Score: 0.005842783840031299
